### Scraping des Produits "Arome & Liquide" (Thème Fruité)
Ce notebook est divisé en deux parties principales pour le scraping des informations liées aux produits de type liquide fruité sur le site Arome & Liquide.

1. Scraping des liens et des informations de base des produits :

L'objectif de cette étape est de collecter les informations générales des produits.
Les informations extraites incluent :Lien du produit, Nom du produit, Petite description, URL de l'image (JPG)

> Ces données sont ensuite sauvegardées dans un CSV.
Extraction des informations détaillées des liquides :

2. À partir du CSV généré dans la première partie, chaque lien produit est exploré pour récupérer des informations plus détaillées sur les liquides.
Les nouvelles informations extraites sont ensuite stockées dans un nouveau CSV.
Ce notebook automatise donc la collecte d'informations sur les liquides fruités en deux étapes successives, permettant de construire une base de données structurée à partir des produits du site Arome & Liquide.

# Import les modules nécéssares


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
from scrapy.crawler import CrawlerProcess
import os


import requests


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import time

il faut aussi l'img 

In [3]:


current_page = 1
data = []
proceed = True

output_folder = 'links'
os.makedirs(output_folder, exist_ok=True)

while proceed and current_page:
    print("Currently scraping page: " + str(current_page))
    url = "https://www.aromes-et-liquides.fr/102-e-liquides-gourmands?page=" + str(current_page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    all_products = soup.find_all("article")

    # Si aucun produit n'est trouvé, arrêter la boucle
    if not all_products:
        proceed = False
    else:
        for product in all_products:
            item = {}
            # Titre du produit
            item['nom_produit'] = product.find("img").attrs["alt"]
            # Lien du produit
            item['lien_produit'] = product.find("a").attrs["href"]
            
            # Récupérer la description du produit
            product_desc = product.find("div", class_="product-desc")
            if product_desc:
                item['Description'] = product_desc.text.strip()
            else:
                item['Description'] = None

            # Récupérer l'image du produit
            product_img = product.find("img", itemprop="image")
            if product_img:
                item['img_produit'] = product_img.get("data-full-size-image-url", product_img.get("src"))
            else:
                item['img_produit'] = None

            data.append(item)

        current_page += 1

# Définir le nom du fichier CSV dans le dossier "links"
csv_file = os.path.join(output_folder, "link_produits_desserts_a&i.csv")

# Sauvegarder les données au format CSV
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ['nom_produit', 'lien_produit', 'Description', 'img_produit']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

print(f"Data successfully saved to {csv_file}")


Currently scraping page: 1
Currently scraping page: 2
Currently scraping page: 3
Currently scraping page: 4
Currently scraping page: 5
Currently scraping page: 6
Currently scraping page: 7
Currently scraping page: 8
Data successfully saved to links\link_produits_desserts_a&i.csv


In [5]:

# Configuration de Selenium
driver = webdriver.Chrome()

# Configuration des chemins
csv_file = "links/link_produits_desserts_a&i.csv"
scrap_folder = 'donnees_scrappes'
output_file = os.path.join(scrap_folder, 'produits_dessert_A&L.csv')
os.makedirs(scrap_folder, exist_ok=True)

# Vérifier et créer le fichier CSV de sortie s'il n'existe pas
if not os.path.exists(output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        fieldnames = ['lien_produit', 'nom_produit', 'Title', 'Description', 'Features', 'img_produit', 'prix_produit']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

# Dictionnaire pour stocker les données des produits
link_data = {}

# Lecture des liens dans le fichier CSV
links = []
try:
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            links.append(row['lien_produit'])
            link_data[row['lien_produit']] = {
                'Description': row['Description'],
                'nom_produit': row['nom_produit'],
                'Features': {},
                'img_produit': row['img_produit'],
                'prix_produit': "N/A",
                'Title': ''  # Nouvelle clé pour le titre
            }
except Exception as e:
    print(f"Erreur lors de la lecture du fichier CSV : {e}")
    exit(1)

# Début du scraping
try:
    for link in links:
        driver.get(link)

        # Attendre que la page soit chargée
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

        # Gestion des cookies
        try:
            accept_cookies_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".butonaccept"))
            )
            accept_cookies_button.click()
            time.sleep(1)
        except Exception:
            pass

        # Gestion de la date de naissance
        try:
            jour_input = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, "jour")))
            mois_input = driver.find_element(By.ID, "mois")
            annee_input = driver.find_element(By.ID, "annee")

            jour_input.send_keys("01")
            mois_input.send_keys("01")
            annee_input.send_keys("1990")
        except Exception:
            pass

        # Extraction avec BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Titre du produit
        try:
            title_h1 = soup.find('h1', itemprop='name')
            if title_h1:
                link_data[link]['Title'] = title_h1.get_text(strip=True)
        except Exception:
            pass

        # Description
        try:
            description_div = soup.find('div', class_='description')
            formatted_description = ""
            if description_div:
                for element in description_div.find_all(['h2', 'h3', 'p']):
                    if element.name in ['h2', 'h3']:
                        formatted_description += f"<{element.name}>{element.get_text(strip=True)}</{element.name}>\n"
                    else:
                        formatted_description += f"<p>{element.get_text(strip=True)}</p>\n"
                link_data[link]['Description'] = formatted_description
        except Exception:
            pass

        # Caractéristiques
        try:
            table = soup.find('table', class_='table-features')
            if table:
                for row in table.find_all('tr'):
                    th, td = row.find('th'), row.find('td')
                    if th and td:
                        link_data[link]['Features'][th.get_text(strip=True)] = td.get_text(strip=True)
        except Exception:
            pass

        # Prix
        try:
            price_div = soup.find('div', class_='current-price')
            if price_div:
                price_span = price_div.find('span', class_='price', itemprop='price')
                if price_span and price_span['content']:
                    link_data[link]['prix_produit'] = price_span['content']
        except Exception:
            pass

        # Sauvegarde directe dans le fichier CSV
        with open(output_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['lien_produit', 'nom_produit', 'Title', 'Description', 'Features', 'img_produit', 'prix_produit'])
            writer.writerow({
                'lien_produit': link,
                'nom_produit': link_data[link]['nom_produit'],
                'Title': link_data[link]['Title'],
                'Description': link_data[link]['Description'],
                'Features': link_data[link]['Features'],
                'img_produit': link_data[link]['img_produit'],
                'prix_produit': link_data[link]['prix_produit']
            })
        print(f"Produit extrait et sauvegardé : {link_data[link]['Title']}")
except Exception as e:
    print(f"Erreur lors du scraping : {e}")
finally:
    driver.quit()

print(f"Les données ont été sauvegardées dans {output_file}.")


Produit extrait et sauvegardé : La Chose Le French Liquide
Produit extrait et sauvegardé : Tutti Frutti 100ml Sweets Chuffed
Produit extrait et sauvegardé : Menthe Chlorophylle Cirkus VDLV
Produit extrait et sauvegardé : Strawberry Candy Floss 100ml Sweets Chuffed
Produit extrait et sauvegardé : Lemon Tart 50ml Dinner Lady
Produit extrait et sauvegardé : La Noisette 200ml Jin & Juice
Produit extrait et sauvegardé : Vanille Extrême PULP
Produit extrait et sauvegardé : Fraise Candy 50ml Sweety Fruits
Produit extrait et sauvegardé : Corne de gazelle PULP
Produit extrait et sauvegardé : Candy Cola Alfaliquid
Produit extrait et sauvegardé : Le Popcorn 200ml Jin & Juice
Produit extrait et sauvegardé : Tribeca Halo
Produit extrait et sauvegardé : Barbe À Papa Alfaliquid
Produit extrait et sauvegardé : Alucard 50ml Ultimate A&L
Produit extrait et sauvegardé : Cinema Reserve Act 1 100ml Clouds of Icarus
Produit extrait et sauvegardé : Famous Eliquid France
Produit extrait et sauvegardé : Macada

In [6]:
df = pd.read_csv("donnees_scrappes/produits_dessert_A&L.csv")
df

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit
0,https://www.aromes-et-liquides.fr/e-liquide-le...,La Chose Le French Liquide,La Chose Le French Liquide,<p>La Chose est une-liquide prêt à l'emploipou...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/54442-thi...,5.9
1,https://www.aromes-et-liquides.fr/e-liquide-ch...,Tutti Frutti 100ml Sweets...,Tutti Frutti 100ml Sweets Chuffed,<p>Le Tutti Frutti 100ml est un e-liquide à bo...,"{'Contenance': '100ml (fiole de 120ml)', 'Taux...",https://assets.aromes-et-liquides.fr/41534-thi...,9.9
2,https://www.aromes-et-liquides.fr/e-liquide-ci...,Menthe Chlorophylle Cirkus...,Menthe Chlorophylle Cirkus VDLV,<p>Le Menthe Chlorophylle par VDLV est un e-li...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/54970-thi...,5.9
3,https://www.aromes-et-liquides.fr/e-liquide-ch...,Strawberry Candy Floss...,Strawberry Candy Floss 100ml Sweets Chuffed,<p>Le Strawberry Candy Floss 100ml est un e-li...,"{'Contenance': '100ml (fiole de 120ml)', 'Taux...",https://assets.aromes-et-liquides.fr/41517-thi...,9.9
4,https://www.aromes-et-liquides.fr/e-liquide-va...,E-liquide Lemon Tart 50ml par Vape Dinner Lady,Lemon Tart 50ml Dinner Lady,<p>Le Lemon Tart est un e-liquide pour cigaret...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/29244-thi...,19.9
...,...,...,...,...,...,...,...
380,https://www.aromes-et-liquides.fr/e-liquide-le...,La Mimi 50ml Les Collègues,La Mimi 50ml Les Collègues,<p>La marque française Liquidarom vous présent...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56179-thi...,19.9
381,https://www.aromes-et-liquides.fr/e-liquide-le...,Le Flambeur 50ml Les Collègues,Le Flambeur 50ml Les Collègues,<p>La marque française Liquidarom vous présent...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56181-thi...,19.9
382,https://www.aromes-et-liquides.fr/e-liquides-g...,Pack Dessert A&L,Pack Dessert A&L,<p>A&L vous propose de débuter la vape tout en...,"{'Contenance': '3 x 10ml', 'Taux PG/VG': '50/5...",https://assets.aromes-et-liquides.fr/55828-thi...,9.9
383,https://www.aromes-et-liquides.fr/e-liquide-sw...,Vap Chef 50ml Swoke,Vap Chef 50ml Swoke,<p>Le e-liquide pour cigarette électronique Va...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56420-thi...,19.9


nb nom_produit unique
824 de 845 lignes 

In [7]:
df['nom_produit'].nunique()

385

nb lien_produit unique
832 de 845 lignes 

In [8]:
df['lien_produit'].nunique()

385

In [9]:
df.shape

(385, 7)

Pas besoin de suprimer les noms non unique car ca peut etre même type de produit mais different contenance 
- ! mais je suprime les lien du produit qui se doublent 

In [7]:
df_non_unique_nom_produit = df[df.duplicated(subset='lien_produit', keep=False)]
df_non_unique_nom_produit

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit
597,https://www.aromes-et-liquides.fr/e-liquide-pe...,Rouge des Bois 60ml Petit...,Rouge des Bois 60ml Petit Nuage,<p>Le Rouge des Bois est un e-liquide à booste...,"{'Contenance': '60ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/49771-thi...,19.9
598,https://www.aromes-et-liquides.fr/e-liquide-et...,Teint de Pêche Récolte...,Teint de Pêche Récolte d'Hiver 50ml La Cueille...,<p>Le Teint de Pêche Récolte d'Hiver 50ml est ...,"{'Contenance': '50ml (fiole de 70ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/51461-thi...,21.9
599,https://www.aromes-et-liquides.fr/e-liquide-va...,Green 50ml Enfer Vape47,Green 50ml Enfer Vape47,<p>Le Green 50ml est un e-liquide pour cigaret...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/49527-thi...,22.9
600,https://www.aromes-et-liquides.fr/e-liquide-pe...,Rouge des Bois 60ml Petit...,Rouge des Bois 60ml Petit Nuage,<p>Le Rouge des Bois est un e-liquide à booste...,"{'Contenance': '60ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/49771-thi...,19.9
601,https://www.aromes-et-liquides.fr/e-liquide-et...,Teint de Pêche Récolte...,Teint de Pêche Récolte d'Hiver 50ml La Cueille...,<p>Le Teint de Pêche Récolte d'Hiver 50ml est ...,"{'Contenance': '50ml (fiole de 70ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/51461-thi...,21.9
602,https://www.aromes-et-liquides.fr/e-liquide-va...,Green 50ml Enfer Vape47,Green 50ml Enfer Vape47,<p>Le Green 50ml est un e-liquide pour cigaret...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/49527-thi...,22.9
711,https://www.aromes-et-liquides.fr/e-liquide-pr...,Pomme Fraise Framboise Salt...,Pomme Fraise Framboise Salt Prestige Fruits,<p>Le Pomme Fraise Framboise Salt est une-liqu...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53566-thi...,4.9
712,https://www.aromes-et-liquides.fr/e-liquide-va...,Blood Sukka 50ml Vampire Vape,Blood Sukka 50ml Vampire Vape,<p>Le Blood Sukka 50ml est fabriqué au Royaume...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/52593-thi...,19.9
713,https://www.aromes-et-liquides.fr/e-liquide-va...,Catapult 50ml Vampire Vape,Catapult 50ml Vampire Vape,<p>L'e-liquide surboosté en arômes et sans nic...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/52592-thi...,19.9
720,https://www.aromes-et-liquides.fr/e-liquide-pr...,Pomme Fraise Framboise Salt...,Pomme Fraise Framboise Salt Prestige Fruits,<p>Le Pomme Fraise Framboise Salt est une-liqu...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53566-thi...,4.9


Créer une nouvelle DataFrame avec des lignes ayant des valeurs uniques dans 'lien_produit'

In [8]:

df_unique_lien = df.drop_duplicates(subset='lien_produit').copy()

df_unique_lien.shape

(836, 7)

uniquement les lignes où 'nom_produit' n'est pas unique

In [9]:
df_non_unique_nom_produit = df_unique_lien[df_unique_lien.duplicated(subset='nom_produit', keep=False)]
df_non_unique_nom_produit

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit
287,https://www.aromes-et-liquides.fr/e-liquide-bi...,Grenadine Framboise Fraise...,Grenadine Framboise Fraise 200ml Biggy Bear,<p>Retrouvez unerecette de grenadine fraîche a...,"{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/56416-thi...,14.50
468,https://www.aromes-et-liquides.fr/e-liquide-fr...,Fruit du Dragon Fruits...,Fruit du Dragon Fruits Bleus Frais Platinium E...,<p>Le Fruit du Dragon Fruits Bleus Frais Plati...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/49393-thi...,17.90
511,https://www.aromes-et-liquides.fr/e-liquide-le...,Fruit du Dragon Fruits...,Fruit du Dragon Fruits Rouges Frais 50ml Le Pe...,<p>Le Fruit du Dragon Fruits Rouges Frais 50ml...,"{'Contenance': '50ml (fiole de 70ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/49647-thi...,19.90
586,https://www.aromes-et-liquides.fr/e-liquide-pu...,Corossol Ananas Fruit du...,Corossol Ananas Fruit du Dragon 50ml Punk Funk...,<p>Le Corossol Ananas Fruit du Dragon est un e...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/46393-thi...,21.90
630,https://www.aromes-et-liquides.fr/e-liquide-pr...,Grenadine Framboise Fraise...,Grenadine Framboise Fraise 50ml Prestige Fruits,<p>Le Grenadine Framboise Fraise est un e-liqu...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/51189-thi...,19.90
652,https://www.aromes-et-liquides.fr/e-liquide-pu...,Pastèque Fraise Myrtille...,Pastèque Fraise Myrtille 50ml Punk Funk Hero,<p>Le Pastèque Fraise Myrtille 50ml est un e-l...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/44517-thi...,21.90
685,https://www.aromes-et-liquides.fr/e-liquide-fr...,Fruit Du Dragon Pastèque...,Fruit Du Dragon Pastèque Citron Vert Emeraude ...,<p>Le Fruit du Dragon Pastèque Citron Vert Eme...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/53441-thi...,17.90
688,https://www.aromes-et-liquides.fr/e-liquide-va...,Heisenberg Cherry Dragon...,Heisenberg Cherry Dragon Fruit Ice Vampire Vape,"<p>Avec le Heisenberg Cherry Dragon Fruit Ice,...","{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/54362-thi...,5.50
698,https://www.aromes-et-liquides.fr/e-liquide-va...,Heisenberg Cherry Dragon...,Heisenberg Cherry Dragon Fruit Ice 100ml Vampi...,<p>Le Heisenberg Cherry Fruit Dragon Ice 100ml...,"{'Contenance': '100ml (fiole de 120ml)', 'Taux...",https://assets.aromes-et-liquides.fr/53704-thi...,24.90
766,https://www.aromes-et-liquides.fr/e-liquide-pr...,Fruit Du Dragon Pastèque...,Fruit Du Dragon Pastèque Citron Vert Salt Pres...,<p>Le Fruit Du Dragon Pastèque Citron Vert Sal...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53556-thi...,4.90


de col Features > cols : Contenance	Taux PG/VG	Origine	Frais / Mentholé	Surboosté en arômes	Saveur

In [10]:
import ast  # Pour convertir la chaîne en dictionnaire
# Fonction pour extraire la valeur d'une clé dans la colonne 'Features'
def extract_feature(features_str, feature_name):
    try:
        # Convertir la chaîne en dictionnaire
        features = ast.literal_eval(features_str)
        return features.get(feature_name, None)  # Retourne la valeur ou None si la clé n'existe pas
    except Exception as e:
        print(f"Erreur lors de l'extraction de {feature_name}: {e}")
        return None

# Liste des caractéristiques que vous voulez extraire
features_list = ['Contenance', 'Taux PG/VG', 'Origine', 'Frais / Mentholé', 'Surboosté en arômes', 'Saveur']

# Extraire les caractéristiques dans des colonnes séparées
for feature in features_list:
    df_unique_lien[feature] = df_unique_lien['Features'].apply(lambda x: extract_feature(x, feature))

In [11]:
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,Contenance,Taux PG/VG,Origine,Frais / Mentholé,Surboosté en arômes,Saveur
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,10ml,50/50,France,Oui,Non,Fruitée
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,10ml,50/50,France,Oui,Non,Fruitée
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,10ml,50/50,France,Non,Non,Fruitée
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,200ml,50/50,France,Oui,Oui,Fruitée
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,10ml,50/50,Royaume-Uni,Oui,Non,Fruitée
...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,https://www.aromes-et-liquides.fr/e-liquide-sw...,Rakam 50ml Saint Flava Swoke,Rakam 50ml Saint Flava Swoke,<p>Le e-liquide pour cigarette électronique Ra...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56185-thi...,19.90,50ml (fiole de 75ml),40/60,France,Non,Oui,Fruitée
843,https://www.aromes-et-liquides.fr/e-liquides-f...,Pack e-liquides du mois -...,Pack e-liquides du mois - Novembre 2024,<p>Voici pour le mois de novembre 2024 un lot ...,"{'Contenance': '3 x 50ml', 'Origine': 'France'...",https://assets.aromes-et-liquides.fr/55829-thi...,45.90,3 x 50ml,None,France,None,Oui,Fruitée
844,https://www.aromes-et-liquides.fr/e-liquide-ri...,Watermelon Freez 50ml Ripe...,Watermelon Freez 50ml Ripe Vapes,<p>La marque originaire des États-Unis Ripe Va...,"{'Contenance': '50ml (fiole de 60ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56418-thi...,19.90,50ml (fiole de 60ml),30/70,USA,Oui,Oui,Fruitée
845,https://www.aromes-et-liquides.fr/e-liquide-fi...,Yamakasi 100ml Fighter Fuel,Yamakasi 100ml Fighter Fuel,<p>La collection Fighter Fuel vous propose un ...,"{'Contenance': '100ml (fiole de 120ml)', 'Taux...",https://assets.aromes-et-liquides.fr/56459-thi...,24.90,100ml (fiole de 120ml),30/70,France,Oui,Oui,Fruitée


In [12]:
# Fonction pour extraire les paragraphes après un <h2> ou <h3> spécifique
def extraire_paragraphes_multiples(description, sections_cibles):
    # Diviser la description en lignes
    lignes = description.splitlines()
    
    # Dictionnaire pour stocker les résultats
    resultats = {section: [] for section in sections_cibles}
    capture = None  # Variable pour indiquer quelle section est en cours de capture

    for ligne in lignes:
        # Si la ligne contient un <h2> ou <h3> avec un texte cible, commencer à capturer
        for section in sections_cibles:
            if ("<h2>" in ligne or "<h3>" in ligne) and section in ligne:
                capture = section
                break  # Pas besoin de vérifier d'autres sections si une correspondance est trouvée
        
        # Si une capture est en cours, ajouter les paragraphes
        if capture and "<p>" in ligne and "</p>" in ligne:
            paragraphe = ligne.replace("<p>", "").replace("</p>", "").strip()
            resultats[capture].append(paragraphe)
        
        # Arrêter la capture à la prochaine balise <h2> ou <h3> non ciblée
        if capture and ("<h2>" in ligne or "<h3>" in ligne) and all(section not in ligne for section in sections_cibles):
            capture = None

    # Joindre les paragraphes de chaque section
    return {section: "\n".join(paragraphes) if paragraphes else None for section, paragraphes in resultats.items()}

# Sections cibles
sections_cibles = ["Qu'est-ce que", "Pourquoi", "Comment", "À qui s'adresse", "De quoi se compose", "Qui est", "Nos conseils", "Quel taux de nicotine", "Conservation"]

# Appliquer la fonction à chaque ligne de la DataFrame
extractions = df_unique_lien['Description'].apply(lambda x: extraire_paragraphes_multiples(x, sections_cibles))

# Ajouter les colonnes à partir des résultats
df_unique_lien['Section_Qu_est_ce_que'] = extractions.apply(lambda x: x["Qu'est-ce que"])  # produit
df_unique_lien['Section_Pourquoi'] = extractions.apply(lambda x: x["Pourquoi"])
df_unique_lien['Section_Comment'] = extractions.apply(lambda x: x["Comment"])
df_unique_lien['Section_A_Qui'] = extractions.apply(lambda x: x["À qui s'adresse"])
df_unique_lien['Section_De_Qui'] = extractions.apply(lambda x: x["De quoi se compose"])
df_unique_lien['Section_Marque'] = extractions.apply(lambda x: x["Qui est"])
df_unique_lien['Section_conseil'] = extractions.apply(lambda x: x["Nos conseils"])
df_unique_lien['Section_taux_nicotine'] = extractions.apply(lambda x: x["Quel taux de nicotine"])
df_unique_lien['Section_Conservation'] = extractions.apply(lambda x: x["Conservation"])



In [13]:
# Fonction pour extraire le premier paragraphe d'une ligne
def extraire_premier_paragraphe(description):
    # Trouver la première occurrence de <p> et </p>
    debut = description.find("<p>")
    fin = description.find("</p>")
    
    # Si les balises <p> et </p> sont trouvées, extraire le texte entre elles
    if debut != -1 and fin != -1:
        return description[debut + 3:fin].strip()  # +3 pour ignorer <p>
    else:
        return None  # Retourner None si aucune balise <p> n'est trouvée

# Appliquer la fonction à la colonne 'Description'
df_unique_lien['info'] = df_unique_lien['Description'].apply(extraire_premier_paragraphe)

In [14]:
df_unique_lien

,lien_produit,nom_produit,Title,Description,Features,img_produit,prix_produit,Contenance,Taux PG/VG,Origine,...,Section_Qu_est_ce_que,Section_Pourquoi,Section_Comment,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,10ml,50/50,France,...,"Le Ragnarok, c'est unerecette fraiche aux goût...","Simple, pratique et efficace, le e-liquide Rag...",Un e-liquide 10ml comme le Ragnarok Ultimate e...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,10ml,50/50,France,...,Le e-liquide Oni ? Unerecetteque l'on ne prése...,Pensé pour tous les vapoteurs qui débutent dan...,"Une fois le taux nicotinique sélectionné, vous...","Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,10ml,50/50,France,...,None,None,None,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,200ml,50/50,France,...,None,None,Ce e-liquide est proposé en version200mlboosté...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge..."
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,10ml,50/50,Royaume-Uni,...,\nLe e-liquide Red Astaire est unproduit prêt ...,\nLe e liquide Red Astaire 10ml est souvent re...,\nPuisqu'il est déjà prêt à être vapoté dans v...,None,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,https://www.aromes-et-liquides.fr/e-liquide-sw...,Rakam 50ml Saint Flava Swoke,Rakam 50ml Saint Flava Swoke,<p>Le e-liquide pour cigarette électronique Ra...,"{'Contenance': '50ml (fiole de 75ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/56185-thi...,19.90,50ml (fiole de 75ml),40/60,France,...,None,None,Le Rakam est proposé en version50mlboostée en ...,None,None,Le fabricant français Swoke doit son succès da...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide pour ciga

col marque , je 

In [78]:
print(df_unique_lien.columns)


Index(['lien_produit', 'nom_produit', 'Description', 'Features', 'img_produit',
       'prix_produit', 'Contenance', 'Taux PG/VG', 'Origine',
       'Frais / Mentholé', 'Surboosté en arômes', 'Saveur',
       'Section_Qu_est_ce_que', 'Section_Pourquoi', 'Section_Comment',
       'Section_A_Qui', 'Section_De_Qui', 'Section_Marque', 'Section_conseil',
       'Section_taux_nicotine', 'Section_Conservation', 'info'],
      dtype='object')


In [15]:
print(df_unique_lien['Section_Marque'].isna().sum())  # Compte les valeurs manquantes


46


In [82]:
import re
# Fonction pour extraire le texte entre <h2>Qui est  et  ?</h2>
def extract_text_between_tags(html_text):
    try:
        # Regex pour capturer le texte entre <h2>Qui est  et  ?</h2>
        result = re.search(r'<h2>Qui est\s*(.*?)\s*\?</h2>', html_text)
        if result:
            return result.group(1)  # Retourne le texte capturé
        return None  # Retourne None si aucun texte n'est trouvé
    except:
        return None  # Gestion des erreurs si le texte est mal formaté

# Appliquer la fonction à la colonne 'Description'
df_unique_lien['brand'] = df_unique_lien['Description'].apply(extract_text_between_tags)

In [ ]:
def detect_marque(row):
    # Vérifie si Section_Marque ou nom_produit est NaN
    if pd.isna(row['Section_Marque']) or pd.isna(row['nom_produit']):
        return None  # Retourne None si une des deux valeurs est NaN
    
    nom_produit_words = row['nom_produit'].split()  # Découpe nom_produit en mots
    for word in nom_produit_words:
        if word in row['Section_Marque']:  # Vérifie si le mot est dans Section_Marque
            return word
    return None  # Retourne None si aucun mot ne correspond

# Appliquer la fonction sur tout le DataFrame
df_unique_lien['marque'] = df_unique_lien.apply(detect_marque, axis=1)


In [88]:
marques_liste = df_unique_lien['marque'].dropna().unique().tolist()
# Fonction pour assigner une marque à une ligne si celle-ci est vide et si un mot de la liste correspond
def assign_marque_if_empty(row, marques_liste):
    if pd.isna(row['marque']):  # Si la colonne 'marque' est vide
        for marque in marques_liste:
            if marque in row['nom_produit']:  # Vérifie si la marque est dans nom_produit
                return marque
    return row['marque']  # Retourne la marque existante si elle n'est pas vide

# Appliquer la fonction pour assigner la marque
df_unique_lien['brand'] = df_unique_lien.apply(assign_marque_if_empty, marques_liste=marques_liste, axis=1)

In [89]:
df_unique_lien

,lien_produit,nom_produit,Description,Features,img_produit,prix_produit,Contenance,Taux PG/VG,Origine,Frais / Mentholé,...,Section_A_Qui,Section_De_Qui,Section_Marque,Section_conseil,Section_taux_nicotine,Section_Conservation,info,marque,brand,m
0,https://www.aromes-et-liquides.fr/e-liquide-ul...,Ragnarok Ultimate A&L,<p>Le e-liquide Ragnarok par A&L Ultimate est ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53645-thi...,5.90,10ml,50/50,France,Oui,...,"Avec un tel format prêt à l'emploi, le e-liqui...",Comme tout e-liquide pour cigarette électroniq...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le e-liquide Ragnarok par A&L Ultimate est un ...,A&L,A&L,A&L
1,https://www.aromes-et-liquides.fr/e-liquide-ul...,Oni Ultimate A&L,<p>Le Oni Ultimate 10ml est un e-liquide pour ...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53646-thi...,5.90,10ml,50/50,France,Oui,...,"Avec un format aussi simple et pratique, le e-...",Chaque produit A&L suit un cahier des charges ...,None,None,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Oni Ultimate 10ml est un e-liquide pour cig...,A&L,A&L,A&L
2,https://www.aromes-et-liquides.fr/e-liquide-al...,A&L Fruits Rouges,<p>Le Fruits Rouges est un e-liquide auxsaveur...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/53641-thi...,4.90,10ml,50/50,France,Non,...,None,None,Arômes et Liquides (A&L) est un fabricant fran...,L'utilisation d'un e-liquide est exclusivement...,Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Fruits Rouges est un e-liquide auxsaveurs d...,A&L,A&L,A&L
3,https://www.aromes-et-liquides.fr/e-liquide-he...,Red Light 200ml Hello Cloudy,"<p>Le Red Light, unesaveur fraîche de baies ro...","{'Contenance': '200ml', 'Taux PG/VG': '50/50',...",https://assets.aromes-et-liquides.fr/45672-thi...,11.34,200ml,50/50,France,Oui,...,None,None,Hello Cloudy est un fabricant français de e-li...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,"Le Red Light, unesaveur fraîche de baies rouge...",200ml,200ml,200ml
4,https://www.aromes-et-liquides.fr/e-liquide-t-...,Red Astaire T-Juice,<p>Le Red Astaire est un e-liquide qui a renco...,"{'Contenance': '10ml', 'Taux PG/VG': '50/50', ...",https://assets.aromes-et-liquides.fr/55065-thi...,4.90,10ml,50/50,Royaume-Uni,Oui,...,None,None,None,None,None,None,Le Red Astaire est un e-liquide qui a rencontr...,Red,Red,Red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,https://www.aromes-et-liquides.fr/e-liquide-fu...,Grok 80ml Furiosa SKINZ,<p>Le Grok est un e-liquide à booster au forma...,"{'Contenance': '80ml (fiole de 100ml)', 'Taux ...",https://assets.aromes-et-liquides.fr/56097-thi...,27.50,80ml (fiole de 100ml),30/70,France,Non,...,None,None,"Depuis 2013, Vape47 vous propose de nombreuses...","De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,Le Grok est un e-liquide à booster au format 8...,Furiosa,Furiosa,Furiosa
841,https://www.aromes-et-liquides.fr/e-liquide-el...,Black Lemon 50ml Creative...,<p>La marque française vous propose une saveur...,"{'Contenance': '50ml (fiole de 70ml)', 'Taux P...",https://assets.aromes-et-liquides.fr/54764-thi...,11.34,50ml (fiole de 70ml),50/50,France,Non,...,None,None,Créé en 2013 à Nieul sur Mer en Charente-Marit...,"De la même façon que pour un e-liquide DIY, on...",Le choix de votre taux de nicotine dépend de v...,Pour une conservation optimale de votre e-liqu...,La marque française vous propose une saveur de...,50ml,50ml,50ml
842,https://www.aromes-et-liquides.fr/e-liquide-sw...,R